I start by importing the CSV files and and using methods that give me a high-level overview of the data sets.

In [1]:
import numpy as np, pandas as pd
#path = 'C:/Users/Glenn Wright/Contacts/Documents/GitHub/homework/'
path = 'C:/Users/M543015/Desktop/GitHub/homework/'
base = pd.read_csv(path+'meps_base_data.csv', sep=',')
meds = pd.read_csv(path+'meps_meds.csv', sep=',')

base.dtypes
base.head()
base.describe()

meds.dtypes
meds.head()
meds.describe()

Unnamed: 0            id  rxStartMonth   rxStartYear         rxNDC  \
count  1.148347e+06  1.148347e+06  1.148347e+06  1.148347e+06  1.148347e+06   
mean   1.240252e+06  5.523596e+07  5.184600e-01  1.172427e+03  2.226109e+10   
std    9.590400e+05  2.728584e+07  4.087907e+00  9.895660e+02  2.834269e+10   
min    1.000000e+00  1.000710e+07 -9.000000e+00 -1.400000e+01 -9.000000e+00   
25%    2.870875e+05  4.063810e+07 -1.000000e+00 -1.000000e+00  1.490472e+08   
50%    1.093262e+06  4.947510e+07 -1.000000e+00  2.000000e+03  5.910385e+08   
75%    2.103602e+06  8.118210e+07  1.000000e+00  2.008000e+03  5.486835e+10   
max    3.336212e+06  8.968810e+07  1.200000e+01  2.011000e+03  9.920707e+10   

         rxQuantity  
count  1.148347e+06  
mean   5.942380e+01  
std    3.702845e+02  
min   -9.000000e+00  
25%    3.000000e+01  
50%    3.000000e+01  
75%    6.800000e+01  
max    1.200000e+05

Next, I create some useful collections - a list of all the diseases, and a lookup table for medications by NDC code.

In [2]:
ndc = meds[['rxNDC','rxName','id']]
ndc = ndc.groupby(['rxNDC','rxName'], as_index=False).count()
ndc = ndc.sort_values(['rxNDC','id'], ascending=[True, False])
ndc = ndc.drop_duplicates()
ndc = dict(ndc[['rxNDC','rxName']].values.tolist())
diseases = base.columns[8:].values.tolist()

There may be issues with duplicate meds.  I would want to go over these with a subject-matter expert.

In [18]:
mednames = meds[['rxNDC','rxName','id']]
mednames = mednames.drop_duplicates()
mednames = mednames.groupby(['rxNDC','rxName'], as_index=False).count()
mednames = mednames.sort_values(['rxName','id'], ascending=[True, False])
print mednames

             rxNDC                          rxName   id
35648  63653117105                     CLOPIDOGREL    1
35653  63653117106                     CLOPIDOGREL    1
12822    603385632      HYDROCHLOROTHIAZIDE (HCTZ)    1
7120     228282011             HYDROCHLOROTHIZAIDE    1
16481  13411011303                         LIPITOR    1
16222  12280013100          (ARMOUR) THYROID (1GR)    1
41608  86227090105                              -9    8
0               -9                              -9    5
16013  11673092740                              -9    4
1               -9               1 ANTI-INFECTIVES   54
15520   8881260148        1.5 INCH 27 GAUGE NEEDLE    1
15337   8290305620                    1/2ML TB SYR    2
38693  66993017520                              10    4
22216  50383023210                              10    2
34160  61314001910                              10    2
513        6351935                              10    1
516        6351936                              

Create a long-format disease table, only for IDs that have definitive "Yes" or "No" diagnoses.

In [ ]:
dis_long = base.melt('id',diseases)
dis_long = dis_long.drop_duplicates()
dis_long = dis_long.loc[dis_long['value'].isin(['Yes','No'])]
dis_long = dis_long.replace(['Yes','No'],[1,0])

Merge the long-format disease table with the meds table and then count medications for each disease.

In [ ]:
medsdd = meds[['id', 'rxNDC']]
medsdd = medsdd.drop_duplicates()
dismeds = pd.merge(dis_long, medsdd,'inner','id')
dismeds.groupby(['variable','rxNDC'], as_index=False).count()
yes = dismeds.loc[dismeds['value']==1][['id','variable','rxNDC']]
sums = yes.groupby(['variable','rxNDC'], as_index=False).count()
sums = sums.sort_values(['variable','id'], ascending=[True, False])

1) What are the most common medications for each disease in the base file?

In [ ]:
results1 = {}
ten = {}
fifty = {}
for disease in diseases:
	vals = sums.loc[sums['variable']==disease]
	ten[disease] = [val[1] for val in vals.values if val[2]>=10]
	fifty[disease] = [val[1] for val in vals.values if val[2]<=50]
	vals = vals[:10].values
	results1[disease] = [(val[1], ndc[val[1]]) for val in vals]

We now have a dictionary that contains lists of the ten most frequent medications for each disease.

What medications are most indicative of each disease?

In [ ]:
import scipy.stats as stats
results2 = {}
for disease in diseases:
	result = []
	dis = dis_long.loc[dis_long['variable']==disease]
	for med in ndc.keys():
		if (med in fifty[disease]):
			medyn = meds[['id','rxNDC']].loc[meds['rxNDC']==med]
			medyn = medyn.drop_duplicates()
			medyn['value2'] = 1
			dm = pd.merge(dis_long,medyn,'left','id')
			dm = dm.fillna(0)[['value','value2','id']]
			dm = dm.groupby(['value','value2'], as_index=False).count().values.tolist()
			if (len(dm)==4):
				table = [[dm[0][2], dm[1][2]],[dm[2][2], dm[3][2]]]
				oddsratio, pvalue = stats.fisher_exact(table)
				if pvalue < 0.01:
					print disease
					print ndc[med]
					print oddsratio
					result.append((oddsratio, med, ndc[med]))

	result = sorted(result)[::-1][:5]
	results2[disease] = result

We now have dictionary that shows, for each disease, the five medications with the highest odds ratios.  Because it takes a long time to calculate the odds ratio for every medication, I restricted the analysis to only medications that occurred fifty or more times for each disease in question.

3) Choose ONE of the diseases and build a model to infer whether that disease is present
from the medications.

In [ ]:
diabetes = dis_long.loc[dis_long['variable']=='diabetesDiagnosed'][['id','value']]
dmeds = meds[['id','rxNDC']].loc[meds['rxNDC'].isin(ten['diabetesDiagnosed'])]
dmeds = dmeds.drop_duplicates()
dmeds['ones'] = 1
dmeds = dmeds.pivot('id','rxNDC')
dmeds = dmeds.fillna(0)
dmeds['id'] = dmeds.index

medids = set([id for id in dmeds['id'].values.tolist()])
diabids = set([id for id in diabetes['id'].values.tolist()])
shared = set([id for id in diabids if id in medids])

data = dmeds.loc[dmeds['id'].isin(shared)]
data = data.sort_values('id').drop('id', axis=1)
data = data.values

target = diabetes.loc[diabetes['id'].isin(shared)]
target = target.sort_values('id')
target = target['value'].values

I chose diabetes, because that is a condition I have analyzed before.  I could try to build a model using both medications and demographics, but the instructions say to use medications so that's what I did.  The first step is to reformat the data so it can be used as input for predictive models in scikit-learn.

Given limited time, I decided to try only two modeling techniques: Logistic regression, because I have used it extensively in the past, and random forests, a technique known to get good results when there are important non-linear effects.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.25, random_state=0)
lrmodel = LogisticRegression()
lrmodel.fit(x_train, y_train)

from sklearn.ensemble import RandomForestClassifier
rfmodel = RandomForestClassifier()
rfmodel.fit(x_train, y_train)

4) Demonstrate that the end user should be confident in the result.

There are many ways to evaluate models.  In my previous work, one of the priorities has usually been to make the model interpretable for physicians and other health care providers.  I have found that a table of the sensitivity and precision for different thresholds is a useful tool.  Also, the ROC curve is an extremely common way of visualizing the success of models.

In [ ]:
lrpredict = lrmodel.predict(x_test)
rfpredict = rfmodel.predict(x_test)

lrscores = lrmodel.score(x_test, y_test)
rfscores = rfmodel.score(x_test, y_test)

from sklearn.metrics import roc_curve, auc, confusion_matrix

lrconfuse = confusion_matrix(lrpredict, y_test)
rfconfuse = confusion_matrix(rfpredict, y_test)

tn, fn, fp, tp = lrconfuse.ravel()
lrsens = float(tp) / (tp+fn)
lrprec = float(tp) / (tp+fp)

tn, fn, fp, tp = rfconfuse.ravel()
rfsens = float(tp) / (tp+fn)
rfprec = float(tp) / (tp+fp)

lrprobs = lrmodel.predict_proba(x_test)[:,1:]
rfprobs = rfmodel.predict_proba(x_test)[:,1:]
import matplotlib.pyplot as plt
fpr, tpr, _ = roc_curve(y_test, lrprobs)
fpr, tpr, _ = roc_curve(y_test, rfprobs)
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
ptable = []
for i in np.arange(0.05, 1.0, 0.05):
	predicts = np.where(lrprobs>=i, 1, 0)
	confuse = confusion_matrix(predicts, y_test)
	tn, fn, fp, tp = confuse.ravel()
	sens = float(tp) / (tp+fn)
	prec = float(tp) / (tp+fp)
	row = (i, sens, prec)
	ptable.append(row) 

5) Can you find any evidence that for the disease you’ve modeled, a certain drug is preferred
by a certain demographic subgroup?

In [ ]:
demog = base.loc[base['diabetesDiagnosed']=='Yes']
demogs = {}
demog['marital'] = demog.married.apply(lambda x: x.replace(' IN ROUND',''))
demogs['race'] = demog.race.unique().tolist()
demogs['marital'] = demog.marital.unique().tolist()
demogs['sex'] = demog.sex.unique().tolist()
demog['agecat'] = pd.cut(demog['age'], [0,18,30,40,50,60,70,120])
demogs['agecat'] = demog.agecat.unique().tolist()

dprefer = {}
top = [row[0] for row in results1['diabetesDiagnosed']]
for demo in demogs:
	for cat in demogs[demo]:
		dm = demog[['id',demo]]
		dm[cat] = np.where(dm[demo]==cat, 1, 0)
		dm = dm.drop_duplicates()[['id',cat]]
		dprefer[cat] = []
		for med in top:
		# for med in fifty['diabetesDiagnosed']:
			md = meds.loc[meds['rxNDC']==med]
			md['value'] = 1
			mg = pd.merge(dm, md, 'left', 'id')
			mg = mg.fillna(0)
			mg = mg[['id', cat, 'value']]
			tb = mg.groupby([cat,'value'], as_index=False).count().values.tolist()
			if (len(tb)==4):
				table = [[tb[0][2], tb[1][2]],[tb[2][2], tb[3][2]]]
				oddsratio, pvalue = stats.fisher_exact(table)
				if pvalue < 0.01:
					dprefer[cat].append((oddsratio, med, ndc[med]))

		dprefer[cat] = sorted(dprefer[cat])

The highest odds ratios for any demographic group I examined were for the 0-18 age category.  This makes intuitive sense - it seems likely that physicians use different guidelines for treating children than adults.  There were many other demographic differences that reached statistical significance, but the effect sizes tended to be much smaller.